In [1]:
import os
import json

image_root = r"D:/CompCars/image"

image_to_model = {}

for make_id in os.listdir(image_root):
    make_path = os.path.join(image_root, make_id)
    if not os.path.isdir(make_path):
        continue

    for model_id in os.listdir(make_path):
        model_path = os.path.join(make_path, model_id)
        if not os.path.isdir(model_path):
            continue

        for year in os.listdir(model_path):
            year_path = os.path.join(model_path, year)
            if not os.path.isdir(year_path):
                continue

            if not year.isdigit():
                continue

            for fname in os.listdir(year_path):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    image_to_model[fname] = {
                        "make_id": int(make_id),
                        "model_id": int(model_id),
                        "year": int(year)
                    }

print("Total mapped images:", len(image_to_model))


Total mapped images: 136299


In [6]:
import pandas as pd

attr = pd.read_csv("D:/CompCars/misc/attributes.txt", sep=r"\s+")
print(attr.head())


   model_id  maximum_speed  displacement  door_number  seat_number  type
0         1            235           1.8            5            5     4
1         2            220           1.8            4            5     3
2         3            228           2.0            4            5     3
3         4            230           2.0            5            5     2
4         5            234           3.0            5            5     2


In [7]:
engine_by_image = {}

for fname, info in image_to_model.items():
    model_id = info["model_id"]

    row = attr[attr["model_id"] == model_id]

    if len(row) == 0:
        continue

    row = row.iloc[0]

    # Ignore invalid rows
    if row["displacement"] == 0:
        continue

    engine_by_image[fname] = {
        "displacement": float(row["displacement"]),
        "max_speed": int(row["maximum_speed"]),
        "doors": int(row["door_number"]),
        "seats": int(row["seat_number"])
    }

print("Images with valid engine specs:", len(engine_by_image))


Images with valid engine specs: 97078


In [8]:
from torchvision.datasets import ImageFolder

train_ds = ImageFolder("../dataset/train")
class_names = train_ds.classes

print("Total CNN classes:", len(class_names))
print(class_names[:10])


Total CNN classes: 429
['Audi_Audi_A1', 'Audi_Audi_A3_hatchback', 'Audi_Audi_A3_sedan', 'Audi_Audi_A4L', 'Audi_Audi_A4_estate', 'Audi_Audi_A5_convertible', 'Audi_Audi_A5_coupe', 'Audi_Audi_A5_hatchback', 'Audi_Audi_A6L', 'Audi_Audi_A7']


In [9]:
engine_db = {}

for class_name in class_names:
    class_dir = os.path.join("../dataset/train", class_name)

    if not os.path.isdir(class_dir):
        continue

    for fname in os.listdir(class_dir):
        if fname not in engine_by_image:
            continue

        info = image_to_model.get(fname)
        if info is None:
            continue

        year = info["year"]
        key = f"{class_name}_{year}"

        if key not in engine_db:
            engine_db[key] = engine_by_image[fname]

In [ ]:
engine_db={}
for class_name in class_names:
    class_dir=os.path.join("..dataset/train",class_name)
    
    if not os.path.isdir(class_dir):
        continue
    
    for fname in os.lsitdir(class_dir):
        if fname not in image_to_model:
            continue
        
        info=image_to_model.get(fname)
        if info is None:
            continue
        
        year=info["year"]
        key=f"{class_name}_{year}"
        
        if key not in engine_db:
            engine_db[key]=engine_by_image[fname]

In [10]:
with open("engine_specs.json", "w") as f:
    json.dump(engine_db, f, indent=2)

print("Engine DB entries:", len(engine_db))


Engine DB entries: 1301
